In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import network_diffusion as nd
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

from abcd import configuration_model, correlations, helpers
from src.loaders.net_loader import load_network

## Reversed parameter for the configuration model

In [ ]:
def compute_statistics(net: nd.MultilayerNetwork, mode: str) -> dict[str, list[dict]]:
    
    print("\tComputing statistics")
    degree_stats, partition_stats, edges_stats = [], [], []

    for la_name, lb_name in helpers.prepare_layer_pairs(net.layers.keys()):
        print("\t\t", la_name, lb_name)
        aligned_layers = correlations.align_layers(net, la_name, lb_name, mode)

        degree_stat = correlations.degrees_correlation(aligned_layers[la_name], aligned_layers[lb_name])
        degree_stats.append({(la_name, lb_name): degree_stat})

        partition_stat = correlations.partitions_correlation(aligned_layers[la_name], aligned_layers[lb_name])
        partition_stats.append({(la_name, lb_name): partition_stat})

        edges_stat = correlations.edges_r(aligned_layers[la_name], aligned_layers[lb_name])
        edges_stats.append({(la_name, lb_name): edges_stat})

    return {
        "degree": degree_stats,
        "partition": partition_stats,
        "edges": edges_stats,
    }

def convert_to_correlation_matrix(statistics: dict[str, list[dict]]) -> dict[str, pd.DataFrame]:
    return {
        "degree": helpers.create_correlation_matrix(statistics["degree"]),
        "partition": helpers.create_correlation_matrix(statistics["partition"]),
        "edges": helpers.create_correlation_matrix(statistics["edges"]),
    }


def find_confoguration_setup(net: nd.MultilayerNetwork) -> pd.DataFrame:
    print("\tFinding configuration setup")
    q = {l_name: configuration_model.get_q(l_graph, net.get_actors_num()) for l_name, l_graph in net.layers.items()}
    degrees = {l_name: configuration_model.get_degrees_stats(l_graph) for l_name, l_graph in net.layers.items()}
    partitions = {l_name: configuration_model.get_partitions_stats(l_graph) for l_name, l_graph in net.layers.items()}
    merged_stats = {l_name: {**degrees[l_name], **partitions[l_name], **{"q": q[l_name]}} for l_name in net.layers}
    return pd.DataFrame(merged_stats)


In [ ]:
import matplotlib.pyplot as plt

from matplotlib.figure import Figure


def save_statistics(statistics: dict[str, pd.DataFrame], net_name: str, out_dir: Path) -> None:
    out_dir.mkdir(exist_ok=True, parents=True)
    for stat_name, stat_df in statistics.items():
        stat_df.to_csv(out_dir / f"{net_name}_{stat_name}.csv")


def plot_statistics(statistics: dict[str, pd.DataFrame], net_name: str) -> Figure:

    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(18, 6), gridspec_kw={"width_ratios": [33, 33, 33, 1]})
    fig.tight_layout(pad=2.5, rect=(0.05, 0.05, 0.95, 0.95))

    helpers.plot_heatmap(statistics["degree"], ax[0], ax[-1], "degrees")
    helpers.plot_heatmap(statistics["partition"], ax[1], ax[-1], "partitions")
    helpers.plot_heatmap(statistics["edges"], ax[2], ax[-1], "edges R")

    fig.suptitle(net_name)

    return fig


In [ ]:
mode = "destructive"
networks = [
    "arxiv_netscience_coauthorship",
    "aucs",
    "cannes",
    "ckm_physicians",
    "eu_transportation",
    "l2_course_net_1",
    "lazega",
    "timik1q2009",
    "toy_network",
]

workdir = Path("correlations")
workdir.mkdir(exist_ok=True, parents=True)

pdf = PdfPages(workdir.joinpath(f"correlations.pdf"))
for net_name in sorted(networks):

    print(net_name)
    _net = load_network(net_name, as_tensor=False)
    net = helpers.prepare_network(_net)  # with this we're sure there's no isolated nodes
    if net.is_directed(): raise ValueError("Only undirected networks can be processed right now!")

    statistics_raw = compute_statistics(net, mode)
    statistics_df = convert_to_correlation_matrix(statistics_raw)
    n = configuration_model.get_nb_actors(net)
    l = configuration_model.get_nb_layers(net)
    configuration_df = find_confoguration_setup(net)

    save_statistics(statistics_df, net_name, workdir / "correlations")
    figure = plot_statistics(statistics_df, f"network: {net_name}, n={n}, l={l}")
    figure.savefig(pdf, format="pdf")
    plt.close(figure)
    configuration_df.to_csv(workdir / f"{net_name}_configuration.csv")

pdf.close()


## Degree sequences

In [28]:
def get_degrees_table(net: nd.MultilayerNetwork) -> pd.DataFrame:
    net_degrees = {}
    for l_name, l_graph in net.layers.items():
        net_degrees[l_name] = dict(l_graph.degree())
    return pd.DataFrame(net_degrees).T

In [ ]:
networks = [
    "arxiv_netscience_coauthorship",
    "aucs",
    "cannes",
    "ckm_physicians",
    "eu_transportation",
    "l2_course_net_1",
    "lazega",
    "timik1q2009",
    "toy_network",
]

workdir = Path("correlations/degree_sequences")
workdir.mkdir(exist_ok=True, parents=True)

for net_name in sorted(networks):

    print(net_name)
    _net = load_network(net_name, as_tensor=False)
    net = helpers.prepare_network(_net)  # with this we're sure there's no isolated nodes
    if net.is_directed(): raise ValueError("Only undirected networks can be processed right now!")

    degrees_table = get_degrees_table(net)
    degrees_table.to_csv(workdir / f"{net_name}_degrees.csv")

arxiv_netscience_coauthorship
